# 대규모 언어 모델로 텍스트 생성하기

<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/hg-mldl2/blob/main/10-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

In [1]:
# 깃허브에서 위젯 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정합니다.
from transformers.utils import logging

logging.disable_progress_bar()

## EXAONE-3.5로 상품 질문에 대한 대답 생성하기

In [2]:
from transformers import AutoTokenizer

exaone_tokenizer = AutoTokenizer.from_pretrained(
    "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct")

In [3]:
from transformers import pipeline

pipe = pipeline(task="text-generation",
                model="LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct",
                tokenizer=exaone_tokenizer,
                device=0, trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct:
- configuration_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct:
- modeling_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Device set to use cuda:0


In [4]:
messages = [
    {"role": "system",
     "content": "너는 쇼핑몰 홈페이지에 올라온 질문에 대답하는 Q&A 챗봇이야. \
                 확정적인 답변을 하지 말고 제품 담당자가 정확한 답변을 하기 위해 \
                 시간이 필요하다는 간단하고 친절한 답변을 생성해줘."},
    {"role": "user", "content": "이 다이어리에 내년도 공휴일이 표시되어 있나요?"}
]

pipe(messages, max_new_tokens=200)

[{'generated_text': [{'role': 'system',
    'content': '너는 쇼핑몰 홈페이지에 올라온 질문에 대답하는 Q&A 챗봇이야.                  확정적인 답변을 하지 말고 제품 담당자가 정확한 답변을 하기 위해                  시간이 필요하다는 간단하고 친절한 답변을 생성해줘.'},
   {'role': 'user', 'content': '이 다이어리에 내년도 공휴일이 표시되어 있나요?'},
   {'role': 'assistant',
    'content': '안녕하세요! 다이어리에 내년의 공휴일이 미리 표시되어 있는지에 대한 정확한 답변을 드리기 위해 잠시 시간이 필요합니다. 제품 담당자께서 확인해 주실 예정이니, 조금만 기다려 주세요. 빠르게 알려드릴게요! 😊'}]}]

In [5]:
pipe(messages, max_new_tokens=200, return_full_text=False)

[{'generated_text': '안녕하세요! 다이어리에 대한 질문 감사합니다. 정확한 정보를 드리기 위해서는 제품 담당자님께서 확인이 필요합니다. 공휴일 정보는 매년 변경될 수 있으니, 가능한 빨리 확인해 주시면 좋겠습니다. 추가로 궁금한 점이 있으시다면 언제든지 물어봐 주세요!'}]

In [6]:
output = pipe(messages, max_new_tokens=200, return_full_text=False,
              do_sample=True)
print(output[0]['generated_text'])

물론이죠! 다이어리에 내년도 공휴일 정보가 포함되어 있는지 확인하려면, 먼저 제품 설명이나 카탈로그를 살펴보시는 게 좋을 것 같아요. 만약 온라인으로 확인하고 계시다면, 해당 제품 페이지의 고객 리뷰나 FAQ 섹션에서도 관련 정보를 찾아볼 수 있을 거예요. 제품 담당자님께 직접 문의하시는 것도 가장 확실한 방법일 것 같아요. 시간이 조금 걸리더라도 정확한 답변을 받으실 수 있을 거예요! 😊


## 토큰 디코딩 전략

### 기본 샘플링

In [7]:
import numpy as np

logits = np.array([1, 2, 3, 4, 100])

In [8]:
from scipy.special import softmax

probas = softmax(logits)
print(probas)

[1.01122149e-43 2.74878501e-43 7.47197234e-43 2.03109266e-42
 1.00000000e+00]


In [9]:
np.random.multinomial(100, probas)

array([  0,   0,   0,   0, 100])

In [10]:
probas = softmax(logits/100)
np.random.multinomial(100, probas)

array([15, 13, 16, 12, 44])

In [11]:
output = pipe(messages, max_new_tokens=200, return_full_text=False,
              do_sample=True, temperature=10.0)
print(output[0]['generated_text'])

하이ệ� 사용자 양반!/question=아니메봇하지만 현재 이와 연관어진 정확하고 최종인답변은저희로 부터 도출시키자 한다면 직접 제조사 담당자게 다시 안내해주시거나 직접 공휴일 표시가 상세페이지상 특정 상품코드 뒤 특정 기능 섹션 여부라도 사전 파악함을 권했숖습니다. 혹시 휴일이나 연도설정 및 해당 목록 기능 포함 관련 특정 디테일 부분으로 알아보면 훨씬 나을 터같네뇨.@reference {Contact_Manager /CustomerCarePageURL}] 이렇게 안내받아가심이 원활해져겠다고 본다는말씀드려요 💙


In [12]:
output = pipe(messages, max_new_tokens=200, return_full_text=False,
              do_sample=True, temperature=0.001)
print(output[0]['generated_text'])

안녕하세요! 다이어리에 내년의 공휴일이 미리 표시되어 있는지에 대해 정확한 답변을 드리기 위해서는 제품 담당자에게 확인이 필요합니다. 현재로선 직접 확인이 어려우니, 저희가 안내드릴 수 있는 방법으로는 고객센터에 연락하시거나, 제품 페이지 내의 문의 게시판을 통해 질문해 보시는 것이 좋을 것 같습니다. 담당자분께서 빠르게 답변해 주실 거예요! 감사합니다.


### top-k 샘플링

In [13]:
output = pipe(messages, max_new_tokens=200, return_full_text=False,
              do_sample=True, top_k=10)
print(output[0]['generated_text'])

안녕하세요! 다이어리에 내년의 공휴일이 미리 표시되어 있는지에 대한 답변을 드리기 위해 잠시 시간을 내서 확인이 필요합니다. 담당자가 빠른 시일 내에 정확한 정보를 제공해 드릴 수 있도록 노력하겠습니다. 감사합니다! 궁금한 점이 더 있으시면 언제든지 알려주세요. 😊


In [14]:
output = pipe(messages, max_new_tokens=200, return_full_text=False,
              do_sample=True, top_k=10, temperature=10.0)
print(output[0]['generated_text'])

죄송하게 들리시네요~ 하지만, 아직 저희 웹팀은 다이어리에 정확히 표시되어있는 각 지역마다의 정확 공휴일 내역 확인 중이라 답변 드리기 어려워요일까요!  빠른 시기에는 반영되리 거라 기대되는데 기다리면 다시 알려줄 수 밖에 absence니다. 문의하신 사항이 빨리 해결될 수 되도록 지속 관리중에 계시고 계시죠?


### top-p 샘플링

In [15]:
output = pipe(messages, max_new_tokens=200, return_full_text=False,
              do_sample=True, top_p=0.9)
print(output[0]['generated_text'])

안녕하세요! 다이어리에 내년도 공휴일 정보가 포함되어 있는지 확인하는 건 좋은 질문이에요. 현재 바로 답변 드리기는 어렵지만, 제품 담당자님께 문의하시면 가장 정확한 정보를 제공받으실 수 있을 거예요. 담당자분께서는 다이어리의 제작 연도와 최신 업데이트 정보를 바탕으로 답변해 주실 거예요. 혹시 직접 확인하실 수 있는 부분이 있다면 거기서도 확인해 보시는 것도 좋을 것 같아요! 감사합니다.


In [16]:
output = pipe(messages, max_new_tokens=200, return_full_text=False,
              do_sample=True, temperature=0.8, top_k=100, top_p=0.9)
print(output[0]['generated_text'])

안녕하세요! 다이어리에 내년도 공휴일 정보가 표시되어 있는지에 대해 궁금하시군요. 현재로선 정확한 답변을 드리기 어렵습니다. 공휴일 정보는 연도별로 변동이 있을 수 있고, 제품 출시 시 정확한 데이터를 반영하기 위해 노력하지만, 혹시라도 최신 정보가 필요하시다면 직접 고객센터에 연락하시거나, 저희 웹사이트 내에 있는 자주 묻는 질문(FAQ) 섹션을 확인해 보시는 것도 좋을 것 같아요. 전문가님께서 신속하게 답변드리실 수 있도록 도와드리겠습니다! 감사합니다.


## GPT-4o로 상품 질문에 대한 대답 생성하기

In [17]:
from openai import OpenAI

client = OpenAI(api_key="OpenAI 키를 입력하세요")

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

In [18]:
print(completion.choices[0].message.content)

안녕하세요! 해당 다이어리에 내년도 공휴일이 표시되어 있는지 확인하기 위해 제품 담당자에게 문의해보겠습니다. 조금만 기다려 주세요! 감사합니다.


In [19]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    top_p=0.9
)
print(completion.choices[0].message.content)

안녕하세요! 문의 주셔서 감사합니다. 다이어리에 내년도 공휴일이 포함되어 있는지 확인해 보도록 하겠습니다. 제품 담당자가 정확한 정보를 드릴 수 있도록 시간이 필요합니다. 조금만 기다려 주세요!


In [20]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=1.8
)
print(completion.choices[0].message.content)

답변을 드리는 데 시간이 필요합니다. 제품 담당자가 확인 후 정확한 정보를 제공하도록 하겠습니다. 좀 더 기다려 주시면 감사하겠습니다.
